In [1]:
{-# LANGUAGE OverloadedStrings #-}

import CIO
import Data.Aeson.Lens
import Control.Lens

In [2]:
get "https://api.github.com/users/nmattia" <&>
    (^.responseBody.key "name"._String)

"Nicolas Mattia"

In [3]:
import Control.Monad.IO.Class
import Data.Time

timeIt :: CIO a -> CIO (NominalDiffTime, a)
timeIt act = do
    start <- liftIO $ getCurrentTime
    res <- act
    stop <- liftIO $ getCurrentTime
    pure (diffUTCTime stop start, res)

In [4]:
import Data.UUID (toText)
import System.Random (randomIO)

uuid <- toText <$> randomIO

In [5]:
timeIt $ getWith (param "dummy" .~ [uuid] $ defaults) "https://api.github.com/users/nmattia" <&>
    (^.responseBody.key "name"._String)

(1.370929802s,"Nicolas Mattia")

In [6]:
timeIt $ getWith (param "dummy" .~ [uuid] $ defaults) "https://api.github.com/users/nmattia" <&>
    (^.responseBody.key "name"._String)

(0.00025088s,"Nicolas Mattia")

In [7]:
import Data.Conduit
import Data.Conduit.Combinators as C

In [8]:
sourceToList $ 
    getAllWith 
        (defaults 
        & param "q" .~ ["language:haskell"] 
        & param "sort" .~ ["stars"]
        & param "per_page" .~ ["5"])
        "https://api.github.com/search/repositories"
    .| awaitForever (C.yieldMany . (
        ^..responseBody
        .key "items"
        .values
        .key "name"
        ._String))
    .| C.take 15

"pandoc"

"shellcheck"

"postgrest"

"purescript"

"compiler"

...

"Functional-Programming"

"Carp"

"write-you-a-haskell"

"ghcjs"

"yesod"